<a href="https://colab.research.google.com/github/Waleed-ikh72/Weekly_Project-9/blob/main/Text_Summarization_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning a Model for Summarization Task

In this task, you will load, preprocess, and fine-tune a T5 model on a dataset of news articles for a summarization task. Follow the steps below carefully.

### Model and Dataset Information

For this task, you will be working with the following:

- **Model Checkpoint**: Use the pre-trained model checkpoint `yalsaffar/mt5-small-Arabic-Summarization` if you face any problem you can use `google-t5/t5-small` but the first one is the correct one for both the model and tokenizer.
- **Dataset**: You will be using the `CUTD/news_articles_df` dataset. Ensure to load and preprocess the dataset correctly for training and evaluation.

**Note:**
- Any additional steps or methods you include that improve or enhance the results will be rewarded with bonus points if they are justified.
- The steps outlined here are suggestions. You are free to implement alternative methods or approaches to achieve the task, as long as you explain the reasoning and the process at the bottom of the notebook.
- You can use either TensorFlow or PyTorch for this task. If you prefer TensorFlow, feel free to use it when working with Hugging Face Transformers.
- The number of data samples you choose to work with is flexible. However, if you select a very low number of samples and the training time is too short, this could affect the evaluation of your work.

## Step 1: Load the Dataset

Load the dataset and split it into training and test sets. Use 20% of the data for testing.

In [ ]:
!pip install transformers datasets

In [ ]:
from transformers import pipeline
from datasets import load_dataset

In [ ]:
# checkpoint = 'yalsaffar/mt5-small-Arabic-Summarization'
checkpoint = 'google-t5/t5-small'

## Step 2: Load the Pretrained Tokenizer

Initialize a tokenizer from the gevin model checkpoint.

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [49]:
ds = load_dataset('CUTD/news_articles_df',split = 'train[:20%]')

In [51]:
ds

Dataset({
    features: ['Unnamed: 0', 'summarizer', 'text'],
    num_rows: 1676
})

In [52]:
ds.features

{'Unnamed: 0': Value(dtype='int64', id=None),
 'summarizer': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None)}

In [53]:
ds[5]

{'Unnamed: 0': 5,
 'summarizer': '\nأعلنت وزارة الشؤون الثقافية في بلاغ لها اليوم الاثنين 14 نوفمبر 2016 أنها وضعت حدا لتكليف إبراهيم لطيف مدير الدورة 27 لأيام قرطاج السينمائية. ودعت الوزارة في ذات البلاغ الهيئة المديرة لتقديم التقريرين المالي والأدبي للدورة المذكورة في أقرب الأجال إلى وزارة الإشراف".',
 'text': 'اعلنت وزاره الشؤون الثقافيه بلاغ اليوم الاثنين انها وضعت حدا لتكليف ابراهيم لطيف مدير الدوره لايام قرطاج السينمائيه ودعت الوزاره البلاغ الهيئه المديره لتقديم التقريرين المالي والادبي للدوره المذكوره اقرب الاجال وزاره الاشراف'}

## Step 3: Preprocess the Dataset

Define a preprocessing function that adds a prefix ("summarize:") to each input if needed and tokenizes the text for the model. The labels will be the tokenized summaries.

In [54]:
#Before: i love football
#after : Summerize: i love football
#[summerize....,simmerize...]

In [71]:
def preprocessing(examples):
  text = ['summerize: '+ text for text in examples['text']]
  text_tokenized = tokenizer(text, padding = True, truncation = True,return_tensors = 'pt')

  summary_tokenized = tokenizer(examples['summarizer'], padding = True, truncation = True, return_tensors = 'pt')

  # Create a dictionary with lists of tensors
  input = {
      'input_ids': text_tokenized['input_ids'],
      'attention_mask_txt': text_tokenized['attention_mask'],#[11,,1,1,1,,1,1,1,,1,1,1,0,0,0,0,0,0,0,0,0]
      'labels': summary_tokenized['input_ids'],
      'attention_mask_summary': summary_tokenized['attention_mask']
  }

  return input

In [56]:
tokenized_ds = ds.map(preprocessing, batched = True, batch_size = None)

Map:   0%|          | 0/1676 [00:00<?, ? examples/s]

In [57]:
tokenized_ds

Dataset({
    features: ['Unnamed: 0', 'summarizer', 'text', 'input_ids', 'attention_mask_txt', 'labels', 'attention_mask_summary'],
    num_rows: 1676
})

In [58]:
tokenized_ds[5]

{'Unnamed: 0': 5,
 'summarizer': '\nأعلنت وزارة الشؤون الثقافية في بلاغ لها اليوم الاثنين 14 نوفمبر 2016 أنها وضعت حدا لتكليف إبراهيم لطيف مدير الدورة 27 لأيام قرطاج السينمائية. ودعت الوزارة في ذات البلاغ الهيئة المديرة لتقديم التقريرين المالي والأدبي للدورة المذكورة في أقرب الأجال إلى وزارة الإشراف".',
 'text': 'اعلنت وزاره الشؤون الثقافيه بلاغ اليوم الاثنين انها وضعت حدا لتكليف ابراهيم لطيف مدير الدوره لايام قرطاج السينمائيه ودعت الوزاره البلاغ الهيئه المديره لتقديم التقريرين المالي والادبي للدوره المذكوره اقرب الاجال وزاره الاشراف',
 'input_ids': [1248,
  1737,
  10,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0

## Step 4: Define the Data Collator

Use a data collator designed for sequence-to-sequence models, which dynamically pads inputs and labels.

In [59]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [60]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True
)

## Step 5: Load the Pretrained Model

Load the model for sequence-to-sequence tasks (summarization).

In [61]:
 #already done above

## Step 6: Define Training Arguments

Set up the training configuration with parameters like learning rate, batch size, and number of epochs.

In [66]:
from transformers import Seq2SeqTrainingArguments
args = Seq2SeqTrainingArguments(
    output_dir = 'output_model',
    evaluation_strategy = 'no',
    learning_rate = 0.005,
    num_train_epochs = 1
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Step 7: Initialize the Trainer

Use the `Seq2SeqTrainer` class to train the model.

In [67]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [68]:
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model = model,
    args = args,                                              #First model
    train_dataset = tokenized_ds,
    data_collator = data_collator
   # label_names = ['labels','attention_mask_summary'],
    #input_names = ['input_ids','attention_mask_txt']
)

## Step 8: Fine-tune the Model

Train the model using the specified arguments and dataset.

In [69]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=210, training_loss=0.2748583294096447, metrics={'train_runtime': 87.6313, 'train_samples_per_second': 19.126, 'train_steps_per_second': 2.396, 'total_flos': 226832859267072.0, 'train_loss': 0.2748583294096447, 'epoch': 1.0})

In [ ]:
# from transformers import Seq2SeqTrainer
# trainer = Seq2SeqTrainer(
#     model = model,
#     args = args,
#     train_dataset = tokenized_ds,                             #Error when using T4
#     data_collator = data_collator
#    # label_names = ['labels','attention_mask_summary'],
#     #input_names = ['input_ids','attention_mask_txt']
# )

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# test sample

text = """
تعد اللغة العربية أقدم اللغات الحية على وجه الأرض، و على اختلاف بين الباحثين حول عمر هذه اللغة؛ لا نجد شكاً في أن العربية التي نستخدمها اليوم أمضت ما يزيد على ألف وستمائة سنة، وقد تكفّل الله - سبحانه و تعالى- بحفظ هذه اللغة حتى يرث الله الأرض ومن عليها، قال تعالى {إنا نحن نزلنا الذكر و إنا له لحافظون}، و مذ عصور الإسلام الأولى انتشرت العربية في معظم أرجاء المعمورة وبلغت ما بلغه الإسلام وارتبطت بحياة المسلمين فأصبحت لغة العلم و الأدب والسياسة و الحضارة فضلاً عن كونها لغة الدين والعبادة.

لقد استطاعت اللغة العربية أن تستوعب الحضارات المختلفة؛ العربية، والفارسية، واليونانية، والهندية، المعاصرة لها في ذلك الوقت، و أن تجعل منها حضارة واحدة، عالمية المنزع، إنسانية الرؤية، وذلك لأول مرّة في التاريخ، ففي ظل القرآن الكريم أصبحت اللغة العربية لغة عالمية، واللغة الأم لبلاد كثيرة. إن أهمية اللغة العربية تنبع من نواحٍ عدّة؛ أهمها: ارتباطها الوثيق بالدين الإسلامي و القرآن الكريم، فقد اصطفى الله هذه اللغة من بين لغات العالم لتكون لغة كتابه العظيم و لتنزل بها الرسالة الخاتمة {إنا أنزلناه قرآنا عربيا لعلكم تعقلون}، و من هذا المنطلق ندرك عميق الصلة بين العربية و الإسلام، كما نجد تلك العلاقة على لسان العديد من العلماء ومنهم ابن تيمية حين قال: " معلوم أن تعلم العربية و تعليم العربية فرضٌ على الكفاية ".

"""

## Step 9: Inference

Once the model is trained, perform inference on a sample text to generate a summary. Use the tokenizer to process the text, and then feed it into the model to get the generated summary.

In [82]:
from transformers import pipeline

new_checkpoint =("content/output_model")
summarizer = pipeline("summarization", model=new_checkpoint, tokenizer=tokenizer)



OSError: content/output_model is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:

summary = summarizer(text, max_length=130, min_length=30)

print(summary)

In [73]:
tokenizer(text, return_tensors = 'pt')
model(tokenizer(text, return_tensors = 'pt'))

AttributeError: 